# 1. Abrir GoogleDrive

In [ ]:
from google.colab import drive
import os
from google.colab import files
import pandas as pd

#Montar o Google Drive no Colab com o nome gdrive

drive.mount('/content/gdrive')

# O comando abaixo mostra qual diretório você está trabalhando momento.
print( os.getcwd() )

Caminho_do_Arquivo = 'gdrive/My Drive/NewaveJuly2023/'
# O comando mostra o que está presente dentro da minha pasta com o deck mais recente do Newave no meu google drive,
# que está dentro do meu drive (conta gmail almmarcato). Mudar para o diretório que vocês criaram.
print( os.listdir(Caminho_do_Arquivo) )



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content
['hidr.dat', 'TERM.DAT', 'CLAST.DAT', 'ARQUIVOS.DAT', 'GTMINPAT.DAT', 'MANUTT.DAT', 'EXPT.DAT', 'SHIST.DAT', 'CURVA.DAT', 'REE.DAT', 'format.tmp', 'MENSAG.TMP', 'EXPH.DAT', 'CONFHD.DAT', 'CONFT.DAT', 'DSVAGUA.DAT', 'CVAR.DAT', 'vazoes.dat', 'MODIF.DAT', 'HIDR.DAT', 'mensag.tmp', 'VAZOES.DAT', 'LOSS.DAT', 'ADTERM.DAT', 'AGRINT.DAT', 'GHMIN.DAT', 'PENALID.DAT', 'CASO.DAT', 'C_ADIC.DAT', 'POSTOS.DAT', 'postos.dat', 'RE.DAT', 'FORMAT.TMP', 'DGER.DAT', 'VAZPAST.DAT', 'PATAMAR.DAT', 'SISTEMA.DAT', 'gerado_123314', 'vazpast.dat', 'cvar.dat', 'ghmin.dat', 'ree.dat', 'ConvNomeArqsDados.log', 're.dat', 'pid.txt', 'alertainv.rel', 'consultafcf.rel', 'engthd.dat', 'vazinat.dat', 'vazthd.dat', 'engcont.dat', 'engfiobac.dat', 'engfiob.dat', 'planej.dat', 'engfio.dat', 'engnat.dat', 'LEITURA.TMP', 'x.log', 'energiab.dat', 'energiaf.dat', 'energiaxf.dat', 'energia

# 2. Realiza a leitura do cortesh

In [ ]:
import numpy as np

def le_cortesh(nome):
  TAM_REG = 11520

  file_cortesh = Caminho_do_Arquivo + nome

  with open(file_cortesh, 'rb') as f:

    # Leitura das informacoes do Primeiro Registro
    versao = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    lrec = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    lrece = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    nsis = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    npre = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    nper = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    npst = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    npea = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    nconf = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    nsim = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    npmc = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    anoi = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    mesi = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    lagmax = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    mecaver = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    nsbm = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    nnsbm = np.fromfile(f, dtype=np.dtype('i4'), count=1)[0]
    BUFFER_SIZE = TAM_REG*4 - 17*4
    THEFT1 = f.read(BUFFER_SIZE)

    # Pula para o proximo registro
    f.seek(TAM_REG * 4, 0)

    # Leitura das Informacoes do Segundo Registro
    nreg = npre + nper + npst
    iptreg = np.fromfile(f, dtype=np.dtype('i4'), count=nreg)
    f.seek((TAM_REG - nreg) * 4, 1)

    # Leitura das Informacoes do Terceiro Registro
    nreg = (nper + 2 * npea) * nsis
    mord = np.fromfile(f, dtype=np.dtype('i4'), count=nreg)
    BUFFER_SIZE = TAM_REG*4 - nreg*4
    THEFT3 = f.read(BUFFER_SIZE)
    #f.seek((TAM_REG - nreg) * 4, 1)

    # Leitura das Informacoes do Quarto Registro
    nreg = npre + nper + npst
    pconf = np.fromfile(f, dtype=np.dtype('i4'), count=nreg)
    f.seek((TAM_REG - nreg) * 4, 1)

    # Leitura das Informacoes do Quinto Registro
    nreg = nper * npmc
    fpeng = np.fromfile(f, dtype=np.dtype('f8'), count=nreg)

    data = f.read()

    cortesh = {
          'versao': versao,
          'lrec': lrec,
          'lrece': lrece,
          'nsis': nsis,
          'npre': npre,
          'nper': nper,
          'npst': npst,
          'npea': npea,
          'nconf': nconf,
          'nsim': nsim,
          'npmc': npmc,
          'anoi': anoi,
          'mesi': mesi,
          'lagmax': lagmax,
          'mecaver': mecaver,
          'nsbm': nsbm,
          'nnsbm': nnsbm,
          'iptreg': iptreg,
          'mord': mord,
          'pconf': pconf,
          'fpeng': fpeng,
          'data': data,
          'theft1': THEFT1,
          'theft3': THEFT3
    }

  return cortesh

# 3. Realiza a Leitura do Cortes

In [ ]:
def le_corte(nome, cortesh, periodo):

  file_name_cortes = Caminho_do_Arquivo + nome

  # Ordem Máxima do PARP
  parp_max = int(np.max(cortesh['mord']))

  # Numero de períodos que possuem cortes registrados no arquivo de cortes
  # numero de períodos pre + numero de periodos de planejamento + numero de periodos pos
  # - mes inicial do periodo de planejamento - 1
  # O ultimo período do estudo não possui cortes (funcao de custo futuro)
  periodos = cortesh['npre'] + cortesh['nper'] + cortesh['npst'] - cortesh['mesi']

  num_reg = periodos * cortesh['nsim'] * 24

  # Estrutura para Armazenamento dos Cortes
  ireg = np.zeros([num_reg], dtype=np.dtype('i4'))
  iter = np.zeros([num_reg], dtype=np.dtype('i4'))
  isim = np.zeros([num_reg], dtype=np.dtype('i4'))
  dummy = np.zeros([num_reg], dtype=np.dtype('i4'))
  rhs = np.zeros([num_reg], dtype=np.dtype('f8'))

  piv = np.zeros([num_reg, cortesh['nsis']  ], dtype=np.dtype('f8'))
  pih = np.zeros([num_reg, cortesh['nsis'] * parp_max], dtype=np.dtype('f8'))
  pig = np.zeros([num_reg, cortesh['nsbm'] * cortesh['npmc'] * cortesh['lagmax']], dtype=np.dtype('f8'))


  cortes = None

  # Lê o arquivo de cortes para a memoria -- vetor numpy
  with open(file_name_cortes, 'rb') as f:

    # Registro para Leitura do Arquivo de COrtes
    tam_reg = int((cortesh['lrec'] - 4 * 4 - 8 * 1)/8)
    struct = np.dtype([('ireg', 'i4'), \
                       ('iter', 'i4'), \
                       ('isim', 'i4'), \
                       ('dummy', 'i4'), \
                       ('rhs', 'f8'), \
                       ('ccorte', 'f8', tam_reg) \
                      ])

    # Leitura do arquivo de Cortes
    cortes = np.fromfile(f, dtype=struct)

  # Inicia p Mapeamento dos Cortes do Periodo
  last_cut = cortesh['iptreg'][periodo-1]

  icut = -1
  while last_cut > 0:
    icut += 1
    # Proximo Corte
    next_cut = int(cortes['ireg'][last_cut-1])

    # Termo Independente
    rhs[icut] = cortes['rhs'][last_cut-1]

    # Coeficiente do Multiplicador de Lagrange das Restricoes de EARM
    ini = 0
    fim = cortesh['nsis']
    piv[icut, :] = cortes['ccorte'][last_cut-1, ini:fim]

    # Coeficiente do Multiplicador de Lagrange das Restricoes de EAF
    ini = fim
    fim = ini + cortesh['nsis'] * parp_max
    pih[icut, :] = cortes['ccorte'][last_cut-1, ini:fim]

    # Coeficientes do Multiplicador de Lagrange das Restricoes de GNL
    ini = fim
    fim = ini + cortesh['npmc'] * cortesh['nsbm'] * cortesh['lagmax']
    pig[icut, :] = cortes['ccorte'][last_cut-1, ini:fim]

    # Atualiza ultimo corte lido
    last_cut = next_cut

  corte = {
      'rhs': rhs,
      'piv': piv,
      'pih': pih,
      'pig': pig
  }

  return corte

# 4. Realiza a escrita do cortesh

In [ ]:
import struct

def escreve_cortesh(nome, cortesh):

  TAM_REG = 11520

  file_cortesh = Caminho_do_Arquivo + nome

  with open(file_cortesh, 'wb') as f:
    #
    # Escreve primeiro registro
    #
    versao = cortesh['versao']
    lrec = cortesh['lrec']
    lrece = cortesh['lrece']
    nsis = cortesh['nsis']
    npre = cortesh['npre']
    nper = cortesh['nper']
    npst = cortesh['npst']
    npea = cortesh['npea']
    nconf = cortesh['nconf']
    nsim = cortesh['nsim']
    npmc = cortesh['npmc']
    anoi = cortesh['anoi']
    mesi = cortesh['mesi']
    lagmax = cortesh['lagmax']
    mecaver = cortesh['mecaver']
    nsbm = cortesh['nsbm']
    nnsbm = cortesh['nnsbm']
    registro = np.array([versao, lrec, lrece, nsis, npre, nper, npst, npea,
                       nconf, nsim, npmc, anoi, mesi, lagmax, mecaver,
                       nsbm, nnsbm]).astype('i4')
    for value in registro:
      f.write(struct.pack("i", value))
    #registro = np.zeros(TAM_REG-17).astype('i4') # Preenche final registro 1
    f.write(cortesh['theft1'])
    #for value in registro:
    #  f.write(struct.pack("i", value))

    #
    # Escreve segundo registro
    #
    nreg = npre + nper + npst
    registro = np.array(cortesh['iptreg']).astype('i4')
    for value in registro:
      f.write(struct.pack("i", value))
    registro = np.zeros(TAM_REG-nreg).astype('i4') # Preenche final registro 2
    for value in registro:
      f.write(struct.pack("i", value))
    #
    # Escreve terceiro registro
    #
    nreg = (nper + 2 * npea) * nsis
    registro = np.array(cortesh['mord']).astype('i4')
    for value in registro:
      f.write(struct.pack("i", value))
    #registro = np.zeros(TAM_REG-nreg).astype('i4') # Preenche final registro 3
    f.write(cortesh['theft3'])
    #for value in registro:
    #  f.write(struct.pack("i", value))

    #
    # Escreve quarto registro
    #
    nreg = npre + nper + npst
    registro = np.array(cortesh['pconf']).astype('i4')
    for value in registro:
      f.write(struct.pack("i", value))
    registro = np.zeros(TAM_REG-nreg).astype('i4') # Preenche final registro 4
    for value in registro:
      f.write(struct.pack("i", value))
    #
    # Escreve quinto registro
    #
    nreg = nper * npmc
    registro = np.array(cortesh['fpeng']).astype('f8')
    for value in registro:
        f.write(struct.pack("d", value))
    #registro = np.zeros(TAM_REG-nreg).astype('f8') # Preenche final registro 5
    #registro = np.zeros(TAM_REG-nreg).astype('f8') # Preenche final registro 5
    f.write(cortesh['data'])
    #for value in registro:
    #    f.write(struct.pack("d", value))

    f.close()

  return

# 5. Realiza leitura completa do arquivo de cortes.



In [ ]:
def le_corte_todo(nome, cortesh):

  file_name_cortes = Caminho_do_Arquivo + nome

  # Ordem Máxima do PARP
  parp_max = int(np.max(cortesh['mord']))

  # Numero de períodos que possuem cortes registrados no arquivo de cortes
  # numero de períodos pre + numero de periodos de planejamento + numero de periodos pos
  # - mes inicial do periodo de planejamento - 1
  # O ultimo período do estudo não possui cortes (funcao de custo futuro)
  periodos = cortesh['npre'] + cortesh['nper'] + cortesh['npst'] - cortesh['mesi']

  num_reg = periodos * cortesh['nsim'] * 24

  # Estrutura para Armazenamento dos Cortes
  ireg = np.zeros([num_reg], dtype=np.dtype('i4'))
  iter = np.zeros([num_reg], dtype=np.dtype('i4'))
  isim = np.zeros([num_reg], dtype=np.dtype('i4'))
  dummy = np.zeros([num_reg], dtype=np.dtype('i4'))
  rhs = np.zeros([num_reg], dtype=np.dtype('f8'))

  piv = np.zeros([num_reg, cortesh['nsis']  ], dtype=np.dtype('f8'))
  pih = np.zeros([num_reg, cortesh['nsis'] * parp_max], dtype=np.dtype('f8'))
  pig = np.zeros([num_reg, cortesh['nsbm'] * cortesh['npmc'] * cortesh['lagmax']], dtype=np.dtype('f8'))


  cortes = None

  # Lê o arquivo de cortes para a memoria -- vetor numpy
  with open(file_name_cortes, 'rb') as f:

    # Registro para Leitura do Arquivo de COrtes
    tam_reg = int((cortesh['lrec'] - 4 * 4 - 8 * 1)/8)
    struct = np.dtype([('ireg', 'i4'), \
                       ('iter', 'i4'), \
                       ('isim', 'i4'), \
                       ('dummy', 'i4'), \
                       ('rhs', 'f8'), \
                       ('ccorte', 'f8', tam_reg) \
                      ])

    # Leitura do arquivo de Cortes
    cortes = np.fromfile(f, dtype=struct)

  return cortes

#6. Escreve arquivo de cortes completo

In [ ]:
def escreve_cortes(nome, cortes):

  file_cortes = Caminho_do_Arquivo + nome

  with open(file_cortes, 'wb') as f:
    cortes.tofile(f)
    f.close()

  return

#7. Programa Principal - Só Leitura

In [ ]:
# O comando abaixo chama a leitura do arquivo de cabeçalhos dos cortes
# parametro de entrada nome do arquivo de cabecalho
cortesh = le_cortesh('cortesh.dat')

# O comando abaixo realiza a leitura dos cortes de um dado periodo
# parametros de entrada
#     nome do arquivo contendo os cortes
#     dicionario de dados contendo as informaçoes dos cortes (função anterior)
#     período no qual vc quer ler os cortes

corte = le_corte('cortes.dat',cortesh,9)

print(corte['piv'])

[[-2.0000000e-05 -0.0000000e+00 -0.0000000e+00 ...  0.0000000e+00
  -1.7289959e+02 -5.2110000e-02]
 [-1.0000000e-05 -9.4268920e+01 -6.8747247e+02 ...  0.0000000e+00
  -2.3305657e+02 -8.2750000e-02]
 [-3.0000000e-05 -0.0000000e+00 -0.0000000e+00 ...  0.0000000e+00
  -2.0631737e+02 -2.8040000e-02]
 ...
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]


# 8. Novo Programa Principal - Testando gravação

In [ ]:
#
# Lê arquivos originais
#
cortesh = le_cortesh('cortesh.dat')
cortes = le_corte_todo('cortes.dat', cortesh)

#
# Escreve arquivos fakes
#
escreve_cortesh('fakeh.dat', cortesh)
escreve_cortes('fake.dat', cortes)

#
# Lê arquivos fakes
#
cortesh_fake = le_cortesh('fakeh.dat')
corte = le_corte('cortes.dat',cortesh,9)
corte_fake = le_corte('fake.dat',cortesh_fake,9)
print(corte['rhs'])
print(corte_fake['rhs'])

[3.34482655e+08 3.28636453e+08 3.38177084e+08 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[3.34482655e+08 3.28636453e+08 3.38177084e+08 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [ ]:
print(cortesh_fake.keys())

dict_keys(['versao', 'lrec', 'lrece', 'nsis', 'npre', 'nper', 'npst', 'npea', 'nconf', 'nsim', 'npmc', 'anoi', 'mesi', 'lagmax', 'mecaver', 'nsbm', 'nnsbm', 'iptreg', 'mord', 'pconf', 'fpeng'])


In [ ]:
print(cortesh_fake['pconf'])
print(cortesh['pconf'])

[ 1  1  1  1  1  1  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18
 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42
 43 44 45 46 47 48 49 50 51 52 53 54 54 54 54 54 54 54 54 54 54 54 54 54
 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54
 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54]
[ 1  1  1  1  1  1  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18
 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42
 43 44 45 46 47 48 49 50 51 52 53 54 54 54 54 54 54 54 54 54 54 54 54 54
 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54
 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54]


In [ ]:
def compare_binary_files(file1_path, file2_path):
    BUFFER_SIZE = 11520  # Tamanho do bloco de leitura
    position = 0

    conta = 1
    with open(file1_path, 'rb') as file1, open(file2_path, 'rb') as file2:

        while True:
            print(conta)
            conta += 1
            data1 = file1.read(BUFFER_SIZE)
            data2 = file2.read(BUFFER_SIZE)

            if data1 != data2:
                for i, (byte1, byte2) in enumerate(zip(data1, data2)):
                    if byte1 != byte2:
                        print(f"Diferença na posição {position + i}: {byte1} != {byte2}")
                return False

            position += len(data1)

            if not data1:
                return True

# Caminhos dos arquivos a serem comparados
file1_path = Caminho_do_Arquivo + 'fakeh.dat'
file2_path = Caminho_do_Arquivo + 'cortes.dat'

if compare_binary_files(file1_path, file2_path):
    print("Os arquivos são idênticos.")
else:
    print("Os arquivos são diferentes.")